In [19]:
pip install google-cloud-vision


Note: you may need to restart the kernel to use updated packages.


In [31]:
import os
import io

from google.cloud import vision
from google.cloud.vision_v1 import types
import http.client

In [32]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'tcnjhack-351c09114ab7.json'

In [33]:
client = vision.ImageAnnotatorClient()

In [34]:
FILE_NAME = 'TestImage.jpeg'

In [35]:
FOLDER_PATH = r'/Users/rajshah/Documents'

In [36]:
with io.open(os.path.join(FOLDER_PATH, FILE_NAME), 'rb') as image_file:
        content = image_file.read()

In [38]:
image = vision.Image(content=content)

In [39]:
response_label = client.label_detection(image=image)
for label in response_label.label_annotations:
    print({"label": label.description, "score": label.score})

{'label': 'Smile', 'score': 0.9465373754501343}
{'label': 'Event', 'score': 0.7408259510993958}
{'label': 'Technology', 'score': 0.7338182926177979}
{'label': 'Public speaking', 'score': 0.7240464687347412}
{'label': 'Teacher', 'score': 0.7200205326080322}
{'label': 'Font', 'score': 0.7125822901725769}
{'label': 'Presentation', 'score': 0.708622932434082}
{'label': 'Room', 'score': 0.6995046138763428}
{'label': 'Blackboard', 'score': 0.6991627812385559}
{'label': 'Art', 'score': 0.6965746879577637}


In [40]:
response_face = client.face_detection(image=image)

face_data = []

for face_detection in response_face.face_annotations:
    d = {
        'confidence': face_detection.detection_confidence,
        'joy': face_detection.joy_likelihood,
        'sorrow': face_detection.sorrow_likelihood,
        'surprise': face_detection.surprise_likelihood,
        'anger': face_detection.anger_likelihood
    }
    print(d)


{'confidence': 0.9723659753799438, 'joy': <Likelihood.VERY_LIKELY: 5>, 'sorrow': <Likelihood.VERY_UNLIKELY: 1>, 'surprise': <Likelihood.VERY_UNLIKELY: 1>, 'anger': <Likelihood.VERY_UNLIKELY: 1>}


In [41]:
response_image = client.image_properties(image=image)

image_data = []

for c in response_image.image_properties_annotation.dominant_colors.colors[:3]:
    d = {
        'color': c.color,
        'score': c.score,
        'pixel_fraction': c.pixel_fraction
    }
    print(d)


{'color': red: 134.0
green: 119.0
blue: 102.0
, 'score': 0.25227710604667664, 'pixel_fraction': 0.20233333110809326}
{'color': red: 97.0
green: 89.0
blue: 84.0
, 'score': 0.1810726821422577, 'pixel_fraction': 0.22126667201519012}
{'color': red: 253.0
green: 248.0
blue: 222.0
, 'score': 0.057022836059331894, 'pixel_fraction': 0.04506666585803032}


In [42]:
response_text = client.text_detection(image=image)

for r in response_text.text_annotations:
    d = {
        'text': r.description
    }
    print(d)


{'text': '%24\n'}
{'text': '%24'}
